In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("D:/Datasets/Amazon_reviews/test.csv")
data.columns=["popularity","reviews_header","review_body"]
data.head()

,popularity,reviews_header,review_body
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...


In [3]:
df=data.iloc[:,0:2]
df.head(10)

,popularity,reviews_header
0,2,One of the best game music soundtracks - for a...
1,1,Batteries died within a year ...
2,2,"works fine, but Maha Energy is better"
3,2,Great for the non-audiophile
4,1,DVD Player crapped out after one year
5,1,Incorrect Disc
6,1,DVD menu select problems
7,2,Unique Weird Orientalia from the 1930's
8,1,"Not an ""ultimate guide"""
9,2,Great book for travelling Europe


In [4]:
df.shape

(399999, 2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399999 entries, 0 to 399998
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   popularity      399999 non-null  int64 
 1   reviews_header  399975 non-null  object
dtypes: int64(1), object(1)
memory usage: 6.1+ MB


In [6]:
y=df.popularity.values
y.shape,y.dtype

((399999,), dtype('int64'))

In [7]:
x=df.iloc[:,1].values
x.shape
x.dtype,x[:5]

(dtype('O'),
 array(["One of the best game music soundtracks - for a game I didn't really play",
        'Batteries died within a year ...',
        'works fine, but Maha Energy is better',
        'Great for the non-audiophile',
        'DVD Player crapped out after one year'], dtype=object))

In [8]:
import re
filtered_rev=[]
for rev in x:
    #X has to be in 1D only
    rev=re.sub('[^a-zA-Z]', ' ', str(rev))
    filtered_rev.append(rev)
x=filtered_rev
x[:5]

['One of the best game music soundtracks   for a game I didn t really play',
 'Batteries died within a year    ',
 'works fine  but Maha Energy is better',
 'Great for the non audiophile',
 'DVD Player crapped out after one year']

In [9]:
review_bag=[]
for data in x:
    data=data.lower()
    data=data.split()
    review_bag.append(data)
x=review_bag
x[:5]

[['one',
  'of',
  'the',
  'best',
  'game',
  'music',
  'soundtracks',
  'for',
  'a',
  'game',
  'i',
  'didn',
  't',
  'really',
  'play'],
 ['batteries', 'died', 'within', 'a', 'year'],
 ['works', 'fine', 'but', 'maha', 'energy', 'is', 'better'],
 ['great', 'for', 'the', 'non', 'audiophile'],
 ['dvd', 'player', 'crapped', 'out', 'after', 'one', 'year']]

In [10]:
import nltk
from nltk.corpus import stopwords

In [11]:
stopword_removed=[]
nonstopword=["not"]
for rev in x:
    short_stopwordremove=[]
    for data in rev:
        if data not in stopwords.words("english"):
            short_stopwordremove.append(data)
        if data in nonstopword:
            short_stopwordremove.append(data)
    stopword_removed.append(short_stopwordremove)
    
x=stopword_removed
x[:5]

[['one', 'best', 'game', 'music', 'soundtracks', 'game', 'really', 'play'],
 ['batteries', 'died', 'within', 'year'],
 ['works', 'fine', 'maha', 'energy', 'better'],
 ['great', 'non', 'audiophile'],
 ['dvd', 'player', 'crapped', 'one', 'year']]

In [12]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
stemmed_bag=[]
for rev in x:
    stem_word=[]
    for data in rev:
        stemword=ps.stem(data)
        if stemword not in stem_word:
            stem_word.append(stemword)
    stemmed_bag.append(stem_word)
x=stemmed_bag
x[:5]

[['one', 'best', 'game', 'music', 'soundtrack', 'realli', 'play'],
 ['batteri', 'die', 'within', 'year'],
 ['work', 'fine', 'maha', 'energi', 'better'],
 ['great', 'non', 'audiophil'],
 ['dvd', 'player', 'crap', 'one', 'year']]

In [13]:
final_review=[]
for rev in x:
    rev=" ".join(rev)
    final_review.append(rev)
    
x=final_review
x[:5],len(x)

(['one best game music soundtrack realli play',
  'batteri die within year',
  'work fine maha energi better',
  'great non audiophil',
  'dvd player crap one year'],
 399999)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [15]:
#.to aaray() is to convert sparse csr_matrix into array
x=cv.fit_transform(x)
x[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
len(cv.get_feature_names_out()),sum(x[0].toarray())

(45009, array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [17]:
sum(x[0].toarray()==1)

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
x.shape

(399999, 45009)

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=47,stratify=y)

In [ ]:
# from sklearn.svm import SVC
# svc=SVC(kernel="rbf",C=1)
# model=svc.fit(x_train,y_train)
# y_pred=model.predict(x_test)
from sklearn.linear_model import LogisticRegression
#max_iter to be increased to eg 500 in logistic regression
lgr=LogisticRegression()
model=lgr.fit(x_train,y_train)
y_pred=model.predict(x_test)

c:\Users\S\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
y_pred,y_test.ravel()

(array([1, 2, 2, ..., 1, 2, 2], dtype=int64),
 array([1, 2, 2, ..., 1, 2, 2], dtype=int64))

In [22]:
lgr.score(x_train,y_train),lgr.score(x_test,y_test)

(0.8470307719711624, 0.818625)